In [41]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn import preprocessing

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import accuracy_score 

from sklearn.cluster import KMeans

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering


import json
import folium
import math
import numbers

In [79]:
class GeoUtil:
        @staticmethod
        def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
            """
            유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
            """
            if x1 is None or y1 is None or x2 is None or y2 is None:
                return None
            assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
            assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
            assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
            assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

            dLon = abs(x2-x1) # 경도 차이
            if dLon >= 180:   # 반대편으로 갈 수 있는 경우
                dLon -= 360   # 반대편 각을 구한다
            dLat = y2-y1      # 위도 차이
            return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

In [80]:
dataset_raw = pd.read_csv("real_final_real.csv")
lpg = pd.read_csv('lpg.csv', encoding='cp949')
dataset = dataset_raw.copy()

# 동별_수소차_비율 구하기.
dataset['동별_수소차_비율'] = dataset['동별_수소차_대수'] / 146

# 동별 승용차 대수 column을 구하고 merge한다. 
dong_car_num = pd.read_csv("original_data_6.csv")
temp = pd.DataFrame(columns = ['자치구', '동', '동별_승용차수'])
temp['자치구'] = dong_car_num['자치구']
temp['동'] = dong_car_num['동']
temp['동별_승용차수'] = dong_car_num['동별_승용차수']
dataset = pd.merge(temp, dataset, on=['자치구', '동'], how='inner')

In [ ]:
dataset.head()

In [9]:
def get_dataframe():
    # 파일 가져오기.
    dataset_raw = pd.read_csv("real_final_real.csv")
    lpg = pd.read_csv('lpg.csv', encoding='cp949')
    dataset = dataset_raw.copy()

    # 동별_수소차_비율 구하기.
    dataset['동별_수소차_비율'] = dataset['동별_수소차_대수'] / 146

    # 동별 승용차 대수 column을 구하고 merge한다. 
    dong_car_num = pd.read_csv("original_data_6.csv")
    temp = pd.DataFrame(columns = ['자치구', '동', '동별_승용차수'])
    temp['자치구'] = dong_car_num['자치구']
    temp['동'] = dong_car_num['동']
    temp['동별_승용차수'] = dong_car_num['동별_승용차수']
    dataset = pd.merge(temp, dataset, on=['자치구', '동'], how='inner')

    # 차의수와 퍼센테이지를 입력받는다. 
    hydrogen_car = int(input("Enter the num of hydrogen car : "))
    percentage = int(input("Enter percentage : "))
    percentage /= 100

    # 수소자동차 대수를 기반으로 hybrid 충전소의 개수 K를 구한다. 
    total_charge_needed = 200 * hydrogen_car # 총 필요 충전량
    total_charge_needed -= 237600
    if total_charge_needed <0:
        total_charge_needed = 0

    num_of_hybrid = math.floor(total_charge_needed / 142560) # hybrid 충전소 개수
    num_of_converged = num_of_hybrid # 융복합충전소 개수
    num_of_mobile = 3 * num_of_converged # 이동식 충전소 개수
    #num_of_off_site = math.floor(total_charge_needed / 57520) # 필요한 off_site 개수

    print("개질기 충전소 개수 : ", num_of_hybrid)
    print("이동식 충전소 개수 : ", num_of_mobile)
    print("원하는 비율 : ", percentage)

    # 차의 수와 동별 수소차의 퍼센테이지를 곱한다. 
    dataset['동별_수소차_대수'] = dataset['동별_수소차_비율'] * hydrogen_car

    # 각 동별 차량의 개수와 퍼센테이지를 곱해 수를 구한다. 
    dataset['승용차수_percentage'] = dataset['동별_승용차수'] * percentage

    # 퍼센테이지 이상인 row를 best_row에 저장한다.

    best_row = []

    for i in range(len(dataset)):
        if dataset['동별_수소차_대수'].iloc[i] >= dataset['승용차수_percentage'].iloc[i]:
            best_row.append(dataset.iloc[i]['동'])

    # best_row에 속하는 동에 있는 row의 index를 구한다. 
    best_row_index = set()

    for i in range(len(dataset)):
        for dong in best_row:
            if dataset['동'].iloc[i] == dong:
                best_row_index.add(i)

    # 새로운 copy()인 best_row_sliced를 만들고 best_row의 index값을 drop해준다.
    best_row_sliced = dataset.copy()
    for i in best_row_index:
        best_row_sliced.drop(index=i, inplace=True)

    # 계산에 필요없는 column을 지워주자. 
    best_row_sliced.drop(columns=['동별_승용차수', '동별_수소차_비율', '승용차수_percentage'], inplace=True)

    # 계산에 필요한 column을 구해주자.
    columns_to_calculate = ['edu_percent', '동별_친환경자동차수', '합계_사업체수', '㎡당 매매평균가', '가구수별_승용차수', '동별_수소차_대수', '택시']

    # 이제 각 column값들을 표준화시켜주자. 
    for column in columns_to_calculate:
        best_row_sliced[column] = (best_row_sliced[column] - min(best_row_sliced[column])) / (max(best_row_sliced[column]) - min(best_row_sliced[column]))

    #best_row_sliced들 가지고 rank를 구하기 위해 sum_1과 interval의 column을 만든다. 
    # sum_1과 column1에 값을 넣는다. 

    best_row_sliced['sum_1'] = list(range(len(best_row_sliced)))
    best_row_sliced['interval_1'] = list(range(len(best_row_sliced)))

    best_row_sliced['sum_1'] = best_row_sliced['edu_percent'] + best_row_sliced['동별_친환경자동차수'] + best_row_sliced['합계_사업체수'] + best_row_sliced['㎡당 매매평균가'] + best_row_sliced['가구수별_승용차수'] + best_row_sliced['동별_수소차_대수'] + best_row_sliced['택시']
    best_row_sliced['interval_1'] = pd.qcut(best_row_sliced['sum_1'], 10, range(1,11))

    # sum_1은 삭제한다. 
    best_row_sliced.drop(columns='sum_1', inplace=True)

    # 원본 dataset에 interval_1의 column을 넣고, 모든 값을 null로 만든 다음 각 best_row_sliced의 값을 interval_1에 넣는다.
    dataset['interval_1'] = None

    for i in range(len(dataset)):
        for j in range(len(best_row_sliced)):
            if dataset.iloc[i]['동'] == best_row_sliced.iloc[j]['동']:
                dataset.loc[i, 'interval_1'] = best_row_sliced.iloc[j]['interval_1']

    # null값에는 가장 높은 값을 준다. 
    for i in range(len(dataset)):
        if pd.isna(dataset.iloc[i]['interval_1']):
            dataset.loc[i, 'interval_1'] = 10

    # 모든 column의 min-max를 구한다. 
    # 이제 각 column값들을 표준화시켜주자. 
    for column in columns_to_calculate:
        dataset[column] = (dataset[column] - min(dataset[column])) / (max(dataset[column]) - min(dataset[column]))

    # sum_2와 interval_2의 coumn을 만든다. 
    dataset['sum_2'] = list(range(len(dataset)))
    dataset['interval_2'] = list(range(len(dataset)))    

    # sum_2에는 필요한 값을 다 더하고
    # interval_2에는 값을 나눈다. 
    dataset['sum_2'] = dataset['edu_percent'] + dataset['동별_친환경자동차수'] + dataset['합계_사업체수'] + dataset['㎡당 매매평균가'] + dataset['가구수별_승용차수'] + dataset['동별_수소차_대수'] + dataset['택시']
    dataset['interval_2'] = pd.qcut(dataset['sum_2'], 10, range(1,11))

    dataset_lpg =  pd.merge(dataset, lpg, on = ['자치구', '동'], how = 'inner')
    dataset_lpg.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)

    dataset_lpg = dataset_lpg.drop_duplicates(subset ="업체명", keep = 'first')

    return dataset_lpg

In [10]:
dataset = get_dataframe()

Enter the num of hydrogen car : 30000
Enter percentage : 10
개질기 충전소 개수 :  40
이동식 충전소 개수 :  120
원하는 비율 :  0.1


In [76]:
data_total_gas_pk = pd.read_csv('data_total_final.csv')

def top_dong(hybrid=2, pk=3):
    data_gas=data_total_gas_pk[['자치구', '동','sum1', '총_점수_interval_10', '수소충전소', '업체명', '동주소', '경도', '위도' ]]
    data_gas.drop_duplicates(subset ="업체명", keep = 'first', inplace=True)
    data_gas.dropna(inplace=True)

    # 수소충전소가 이미 있는 지역은 제외
    indexexist = data_gas[(data_gas['자치구'].isin(data_gas[(data_gas['수소충전소']==1)].자치구))].index
    data_gas.drop(indexexist, inplace=True)

    # 중첩분석 8등급 이상인 주유소만 가져오기 
    data_gas=data_gas[(data_gas['총_점수_interval_10']>7)]
    data_gas.reset_index(drop=True, inplace=True)

    # 거리구하기

    import numbers
    import math

    class GeoUtil:
        @staticmethod
        def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
            """
            유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
            """
            if x1 is None or y1 is None or x2 is None or y2 is None:
                return None
            assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
            assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
            assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
            assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

            dLon = abs(x2-x1) # 경도 차이
            if dLon >= 180:   # 반대편으로 갈 수 있는 경우
                dLon -= 360   # 반대편 각을 구한다
            dLat = y2-y1      # 위도 차이
            return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)
    
    # 거리 2dim넣기
    dist = [[0 for i in range(len(data_gas))] for j in range(len(data_gas))]

    for i in range(len(data_gas)):
        for j in range(len(data_gas)):
            dist[i][j] = GeoUtil.get_euclidean_distance(data_gas.iloc[i,8],data_gas.iloc[i,7],data_gas.iloc[j,8],data_gas.iloc[j,7])
  
    # 군집분석하기
    from sklearn.cluster import AgglomerativeClustering

    cluster = AgglomerativeClustering(n_clusters=hybrid, affinity='euclidean', linkage='single')
    cluster.fit_predict(dist)

    data_gas['sing_clus']=pd.DataFrame(cluster.fit_predict(dist))

    sing_top_dong=data_gas.groupby(['sing_clus','자치구','동','업체명']).agg({'sum1':'mean','총_점수_interval_10':'mean'})
    
    sing_index=sing_top_dong.sort_values(by='sum1', ascending=False).groupby('sing_clus').head(1)
    print(sing_index)
    
    # top 업체명 list 뽑기
    top_lpg = sing_index.index.get_level_values('업체명').tolist()
    top_data = data_gas[(data_gas.업체명.isin(top_lpg))]
    top_data.reset_index(drop=True, inplace=True)

    # 지도에 보이기 
    geo_path = "geoData_copy.json"
    geo_str = json.load(open(geo_path, encoding="utf-8"))

    seoul_map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5, tiles="cartodbpositron")
    seoul_map.choropleth(geo_data=geo_str, 
                         data = data_total_gas_pk ,
                         columns=['동', 'sum1'],
                         fill_color='RdPu',  # GnBu
                         key_on='properties.name',
                         highlight=True,
                         fill_opacity=0.5,
                         line_opacity=1,
                         legend_name="sum1")

    # 양재 수소충전소를 찍어보자.
    folium.Marker(
      location=[37.4686275,127.0319203],
      popup='양재 수소 충전소 위치.',
      icon=folium.Icon(color='red',icon='star')
    ).add_to(seoul_map)

    # 상암 수소 충전소를 찍어보자.
    folium.Marker(
      location=[37.5687204,126.8761933],
      popup='상암 수소 충전소 위치.',
      icon=folium.Icon(color='red',icon='star')
    ).add_to(seoul_map)

    # 국회 수소 충전소를 찍어보자.
    folium.Marker(
      location=[37.5311111,126.9132037],
      popup='2019년 완공 예정 국회 수소 충전소 위치.',
      icon=folium.Icon(color='blue',icon='star')
    ).add_to(seoul_map)

    # 강동구 상일충전소를 찍어보자.
    folium.Marker(
      location=[37.5392486,127.128611],
      popup='2019년 완공 예정 강동구 상일충전소 위치',
      icon=folium.Icon(color='blue',icon='star')
    ).add_to(seoul_map)

    # 추천 lpg 주유소 보이기
    for i in range(len(top_data)):
        folium.Marker(
            location=top_data.loc[i,['경도','위도']],
            popup=top_data.loc[i,['동']],
            icon=folium.Icon(color='lightgray',icon='home')
        ).add_to(seoul_map)
    seoul_map
    seoul_map.save("seoul_map_lpg.html")
    
    ##################################################
    # 주차장 데이터 합치기
    df_parking=data_total_gas_pk[[ '자치구', '동', 'sum1', 'sum_clus', '총_점수_interval_10',  '수소충전소',
                       '주차장명', '주차장구분', '주차장유형', '주차구획수', '주차급지구분', '주차부제시행구분', '주차운영요일',
           '평일운영시작시각', '평일운영종료시각', '토요일운영시작시각', '토요일운영종료시각', '주차요금정보', '주차기본시간',
           '주차기본요금', '주차경도', '주차위도' ]]
    df_parking.drop_duplicates(subset ="주차장명", 
                         keep = 'first', inplace=True)

    # 중첩 6등급 이상인 애들만 가져오기
    data_parking = df_parking[(df_parking.총_점수_interval_10>5)]

    data_parking.dropna(inplace=True)

    # 수소 충전소 있는 애들 빼기
    indexexist = data_parking[(data_parking['자치구'].isin(data_parking[(data_parking['수소충전소']==1)].자치구))].index
    data_parking.drop(indexexist, inplace=True)

    # 수소 충전소 개질기 추천된 애들 빼기
    data_parking = data_parking[~(data_parking.자치구.isin(top_data.자치구))]

    # 개질기 와의 거리 군집
    dist = [[0 for i in range(len(data_parking))] for j in range(len(top_data))]
    
    top_data.reset_index(drop=True, inplace=True)
    data_parking.reset_index(drop=True, inplace=True)

    for i in range(len(top_data)):
        for j in range(len(data_parking)):
            dist[i][j] = GeoUtil.get_euclidean_distance(top_data.loc[i,'위도'],top_data.loc[i,'경도'],data_parking.loc[j,'주차경도'],data_parking.loc[j,'주차위도'])

    dis= pd.DataFrame(dist).T
    
    # 군집구하기 
    from sklearn.cluster import AgglomerativeClustering

    cluster = AgglomerativeClustering(n_clusters=hybrid * pk, affinity='euclidean', linkage='average')
    cluster.fit_predict(dis)

    data_parking['avg_clus']=pd.DataFrame(cluster.fit_predict(dis))

    comp_top_parking=data_parking.groupby(['avg_clus','자치구','동','주차장명']).agg({'sum1':'mean','총_점수_interval_10':'mean'})

    comp_index=comp_top_parking.sort_values(by='sum1', ascending=False).groupby('avg_clus').head(1)
    print(comp_index)
    
    # top 주차장 애들 뽑기
    top_pk_index = comp_index.index.get_level_values('주차장명').tolist()

    top_parking = data_parking[(data_parking.주차장명.isin(top_pk_index))]
    top_parking.reset_index(drop=True, inplace=True)

    # 지도에 나타내기
    for i in range(len(top_parking)):
        folium.Marker(
            location=top_parking.loc[i,['주차위도','주차경도']],
            popup=top_parking.loc[i,['주차장명']],
            icon=folium.Icon(color='gray')
        ).add_to(seoul_map)

    seoul_map.save("seoul_map.html")
    seoul_map

In [77]:
top_dong(8,3)

                                       sum1  총_점수_interval_10
sing_clus 자치구  동    업체명                                      
1         강남구  세곡동  (주)수서에너지      18.364857                10
7         강남구  논현2동 (주)상현상사       10.790736                10
3         송파구  오금동  오금가스충전소        5.464928                10
0         영등포구 문래동  (주)세지          4.918385                 9
2         양천구  신정3동 수도에너지(주)       4.723433                 9
4         동작구  대방동  동일석유(주)동주충전소   3.624785                 9
6         은평구  진관동  은평서부충전소        2.014596                 8
5         동대문구 장안1동 장안동가스충전소       0.354721                 8
                                     sum1  총_점수_interval_10
avg_clus 자치구 동       주차장명                                  
9        구로구 구로2동    하늘공원 주차장    3.805830                 8
11       구로구 구로3동    구로3동 마을공동   2.279064                 6
0        구로구 구로3동    구로디지털단지역    2.279064                 6
22       구로구 구로5동    신도림역 환승     2.059403                 8
1        구로구 구로5동   

In [78]:
data_total_gas_pk[(data_total_gas_pk.주차장명 == '해공공영주차장' )].수소충전소

584    0
Name: 수소충전소, dtype: int64